### **Daily Coding Problem: Problem #28 [Medium] - Palantir**

```
Write an algorithm to justify text. Given a sequence of words and an integer line length k, return a list of strings which represents each line, fully justified.

More specifically, you should have as many words as possible in each line. There should be at least one space between each word. Pad extra spaces when necessary so that each line has exactly length k. Spaces should be distributed as equally as possible, with the extra spaces, if any, distributed starting from the left.

If you can only fit one word on a line, then you should pad the right-hand side with spaces.

Each word is guaranteed not to be longer than k.

For example, given the list of words ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"] and k = 16, you should return the following:
```
```python
["the  quick brown", # 1 extra space on the left
"fox  jumps  over", # 2 extra spaces distributed evenly
"the   lazy   dog"] # 4 extra spaces distributed evenly
```

### Dinamic Programming Steps###
0. **Greedy approach:**
    * Maximize the current line. 
    * Problem: It may not maximize the overall lines.


1. **Subproblems Definition:**
    * **Task:** Trying to fit as many words as possible in each line (== minimize the number of blank spaces).
    * **Question:** For current word `i`, where should I start the next line? (in which of the j remaining words?).
    * **# Subproblems:** for every item I'll ask the previous question, so **n subproblems (n=#words)**


2. **Guess and define the Number of choices:**
    * **Guess:** Refers to answer the question above. 
    * **# guesses (choices):** For current word i I have n - i choices to start the new line (remaining words). **Worst Case: n**


3. **Recurrence Relationship:**
    * **Approach**:
      
      In order to fit the maximum amount of words we are going to work with its complementary. Minimizing blank spaces.  
      Punishing badness, aka, unnecesary white spaces per line.  
      Given that the total sum of white spaces per configuration could be the same, we could take the sum of the square roots of white spaces per line. In case of latex, the use the sum of cubic roots.
    * **Model:** 
      
      Cost of starting line with at word **i** = Cost of starting next line at word **j** + badness(**i to j**).

    ```python
    # k = line lenght
    # words = array of words
    # w = total length of words in a line counting valid white spaces
    # DP[i] = Cost of starting line with at word i
    w = lambda i,j: sum(len(words[i])) for i in range(i, len(words)+j)
    badness = lambda i,j: pow(k - w(i,j), 3) if k - w(i,j) >= 0 else inf
    DP[i] = min(DP[j] + badness(i,j) for j in range(i, len(words) + j) if j fits in line)
    ```


4. **Topological Order:**
    * Depends on the subproblem relationship order.  
      **Forward (ascending):** Current answer depends from before answers. i -> j -> ...  
      **Backward (descending):** Current answer depends from following answers. j -> i -> ...  
      In this case we have a forward relationship. The current line badness (number of unnecesary white spaces) depends on which word will start the next line.


5. **Solving Original Problem:**
    * In order to solve the original problem is necesary to save the least cost starting word at each index.  
    So, for every index in the list of words (for every word) I need to save the least cost next starting word position.  
    For example, if for the first index (first word) I calculate the index of least cost next line starting word I would be able to solve the original problem by assembling lines with the best starting words.

In [3]:
from IPython.core.debugger import set_trace

In [45]:
def space_words_in_line(words, page_width):
    
    line = ''
    words_total_len = sum(len(word) for word in words)
    number_of_dividers = (len(words) - 1)
    total_len = words_total_len + number_of_dividers
    remaining_len = page_width - total_len
    dividers = [' ' for x in range(number_of_dividers)]
    
    for x in range(remaining_len):
        index = x%number_of_dividers
        dividers[index] += ' '
    
    dividers.append('\n')
    
    for x in range(len(words)):
        line += words[x] + dividers[x]
    
    return line


Each value of the `words_least_cost_indices_memo` represents `the index of the least cost next line starting word` at that index. 

In [50]:
# calculating overall cost

def justify_text_helper(starting_word_index, words, page_width, words_least_cost_indices):

    # base case
    if starting_word_index >= len(words): 
        return 0

    # checking memo
    if words_least_cost_indices[starting_word_index] != None:
        return words_least_cost_indices[starting_word_index]

    # divide
    word_fits = True
    words_traversed = False
    words_in_line_guesses = []
    line_len = len(words[starting_word_index])
    next_starting_word_index = starting_word_index + 1

    while word_fits and not words_traversed:
        next_word_starting_cost = justify_text_helper(next_starting_word_index, words, page_width, words_least_cost_indices)
        current_cost = pow(page_width - line_len, 3)
        words_in_line_guesses.append(next_word_starting_cost + current_cost)
        
        if next_starting_word_index < len(words):
            line_len += len(words[next_starting_word_index]) + 1
            if line_len > page_width:
                word_fits = False
        else:
            words_traversed = True

        next_starting_word_index += 1

    # conquer
    minimum_cost = min(words_in_line_guesses)

    # memoizing
    best_next_starting_word_index = starting_word_index + 1 + words_in_line_guesses.index(minimum_cost)
    words_least_cost_indices[starting_word_index] = best_next_starting_word_index

    return minimum_cost

def justify_line_words(words, page_width):
    starting_word_index = 0
    total_words = len(words)
    words_least_cost_indices_memo = [None]*total_words
    overall_cost = justify_text_helper(starting_word_index, words, page_width, words_least_cost_indices_memo)
    
    print('overall_minimum_cost', overall_cost)
    print('words_least_cost_indices_memo', words_least_cost_indices_memo, '\n')
    
    justify_text = ''
    starting_word_index = 0
    next_starting_word_index = words_least_cost_indices_memo[starting_word_index]

    while next_starting_word_index <= total_words:

        line = words[starting_word_index : next_starting_word_index]
        justify_text += space_words_in_line(line, page_width)

        if next_starting_word_index >= total_words: break
        starting_word_index = next_starting_word_index
        next_starting_word_index = words_least_cost_indices_memo[next_starting_word_index]

    print(justify_text)


In [51]:
justify_line_words(["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"], 16)

overall_minimum_cost 7
words_least_cost_indices_memo [3, 4, 5, 6, 7, 8, 9, 9, 9] 

the  quick brown
fox  jumps  over
the   lazy   dog

